In [1]:
import time
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
%%time
dataset = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.data', names=["user_id", "item_id", "rating", "timestamp"], sep="\t")
uq_users = np.sort(dataset.user_id.unique().tolist())
uq_items = np.sort(dataset.item_id.unique().tolist())
n_users = len(uq_users)
n_items = len(uq_items)

CPU times: user 47.8 ms, sys: 15.9 ms, total: 63.7 ms
Wall time: 303 ms


In [3]:
topk = 10
rank_list = [i+1 for i in range(topk)]

# User2User

In [4]:
%%time
user_data = csr_matrix((dataset.rating, (dataset.user_id, dataset.item_id)))
user_similarity = cosine_similarity(user_data)

CPU times: user 80.2 ms, sys: 4.1 ms, total: 84.3 ms
Wall time: 85.3 ms


In [5]:
user_similarity[1:5, 1:5]

array([[1.        , 0.16693098, 0.04745954, 0.06435782],
       [0.16693098, 1.        , 0.11059132, 0.17812119],
       [0.04745954, 0.11059132, 1.        , 0.34415072],
       [0.06435782, 0.17812119, 0.34415072, 1.        ]])

In [6]:
%%time
df_similar_user_list = pd.DataFrame(columns=['base_user_id', 'similar_user_id', 'cosine_score', 'rank'])
for user_id in uq_users:
  similar_score = user_similarity[user_id]
  df_similar_user = pd.DataFrame()
  df_similar_user['base_user_id'] = [user_id] * topk
  df_similar_user['similar_user_id'] = np.argsort(similar_score)[::-1][1:topk+1]
  df_similar_user['cosine_score'] = np.sort(similar_score)[::-1][1:topk+1]
  df_similar_user['rank'] = rank_list
  df_similar_user_list = df_similar_user_list.append(df_similar_user, ignore_index=True)

CPU times: user 6.02 s, sys: 41.9 ms, total: 6.06 s
Wall time: 7.4 s


In [7]:
df_similar_user_list

,base_user_id,similar_user_id,cosine_score,rank
0,1,916,0.569066,1
1,1,864,0.547548,2
2,1,268,0.542077,3
3,1,92,0.540534,4
4,1,435,0.538665,5
...,...,...,...,...
9425,943,276,0.498243,6
9426,943,709,0.493219,7
9427,943,586,0.491972,8
9428,943,472,0.488882,9


# Item2Item

In [8]:
%%time
item_data = csr_matrix((dataset.rating, (dataset.item_id, dataset.user_id)))
item_similarity = cosine_similarity(item_data)

CPU times: user 166 ms, sys: 19.7 ms, total: 185 ms
Wall time: 308 ms


In [9]:
item_similarity[1:5, 1:5]

array([[1.        , 0.40238218, 0.33024479, 0.45493792],
       [0.40238218, 1.        , 0.27306918, 0.50257077],
       [0.33024479, 0.27306918, 1.        , 0.32486639],
       [0.45493792, 0.50257077, 0.32486639, 1.        ]])

In [10]:
%%time
df_similar_item_list = pd.DataFrame(columns=['base_item_id', 'similar_item_id', 'cosine_score', 'rank'])
for item_id in uq_items:
  similar_score = item_similarity[item_id]
  df_similar_item = pd.DataFrame()
  df_similar_item['base_item_id'] = [item_id] * topk
  df_similar_item['similar_item_id'] = np.argsort(similar_score)[::-1][1:topk+1]
  df_similar_item['cosine_score'] = np.sort(similar_score)[::-1][1:topk+1]
  df_similar_item['rank'] = rank_list
  df_similar_item_list = df_similar_item_list.append(df_similar_item, ignore_index=True)

CPU times: user 11 s, sys: 116 ms, total: 11.1 s
Wall time: 14.2 s


In [11]:
df_similar_item_list

,base_item_id,similar_item_id,cosine_score,rank
0,1,50,0.734572,1
1,1,181,0.699925,2
2,1,121,0.689786,3
3,1,117,0.664555,4
4,1,405,0.641322,5
...,...,...,...,...
16815,1682,1268,0.266207,6
16816,1682,1113,0.248069,7
16817,1682,1428,0.231455,8
16818,1682,960,0.229794,9
